In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, smart_resize
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import cv2
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.metrics import AUC
from tqdm.auto import tqdm
sns.set_style('darkgrid')
pd.set_option("display.max_columns", None)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
#train_dir= '../input/resized-plant2021/img_sz_384'
train_dir= "../input/plant-pathology-2021-fgvc8/train_images"
test_dir =  '../input/plant-pathology-2021-fgvc8/test_images'
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
train['labels'] = train['labels'].apply(lambda string: string.split(' '))

In [ ]:
s = list(train['labels'])
mlb = MultiLabelBinarizer()
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train.index)
print(trainx.columns)

In [ ]:
labels = pd.concat([train['image'], trainx], axis=1)
labels.head()

In [ ]:
train_image_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    horizontal_flip=True
    
)
test_image_datagen = ImageDataGenerator(
    rescale=1/255.0 
)

In [ ]:
labels_train,labels_test= train_test_split(labels, test_size=0.1, random_state=0)

In [ ]:
IMAGE = (256, 256)
BATCH_SIZE = 64

train_data = train_image_datagen.flow_from_dataframe(
    dataframe=labels_train,
    directory= '../input/resized-plant2021/img_sz_512',
    x_col="image",
    y_col=labels_train.columns.tolist()[1:],
    color_mode="rgb",
    target_size = IMAGE,
    class_mode="raw",
    #class_mode="categorical",
    #subset = "training",
    batch_size=BATCH_SIZE,
    seed=0,
    shuffle=False
)

test_data = test_image_datagen.flow_from_dataframe(
    dataframe=labels_test,
    directory= '../input/resized-plant2021/img_sz_512',
    x_col="image",
    y_col=labels_test.columns.tolist()[1:],
    color_mode="rgb",
    target_size = IMAGE,
    class_mode="raw",
    #class_mode="categorical",
    #subset = "validation",
    batch_size=BATCH_SIZE,
    seed=0,
    shuffle=False
)

# Modeling

In [ ]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = tf.keras.applications.ResNet50(include_top=False)(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(6, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs, outputs)
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras.models import load_model
model.load_weights('../input/newdataresnet50/resnet50image512 (1).h5')

In [ ]:
import pandas as pd
train_log = pd.read_csv('../input/train-history-resnet50/train_history_resnet50 (1).csv')
train_log.head()

In [ ]:
import matplotlib.pyplot as plt

F1_score = train_log['f1_score']
val_F1_score = train_log['val_f1_score']
loss = train_log['loss']
val_loss = train_log['val_loss']
epochs = range(1, len(F1_score) + 1)

plt.plot(epochs, F1_score)
plt.plot(epochs, val_F1_score)
plt.title('ResNet50 F1-score Curve')
plt.ylabel('F1-score')
plt.xlabel('epoch')
plt.legend(['train','val'],loc = 'upper left')
plt.xlim(0,29)
plt.show()

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('ResNet50 Loss Curve')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'],loc = 'upper left')
plt.xlim(0,29)
plt.show()

**模型预测**

In [ ]:
preds = model.predict(test_data, steps=31, verbose=1)

> **计算其他指标值**

In [ ]:
y_true=labels_test.iloc[:,1:].to_numpy()
y_pred=(preds>0.5)*1

In [ ]:
np.sum(abs(y_true-y_pred))

**计算6类标签中假阳性（FP），假阴性（FN），真阳性（TP），真阴性（TN）数量**

In [ ]:
fp=np.logical_and(y_true==0, y_pred==1)*1
fn=np.logical_and(y_true==1, y_pred==0)*1

tp=np.logical_and(y_true==1, y_pred==1)*1
tn=np.logical_and(y_true==0, y_pred==0)*1

In [ ]:
print(fp.sum(axis=0))
print(fn.sum(axis=0))
print(tp.sum(axis=0))
print(tn.sum(axis=0))

**计算6类标签每类的F1-score、precision和recall值**

In [ ]:
precision=tp.sum(axis=0)/ (tp.sum(axis=0)+fp.sum(axis=0))
recall=tp.sum(axis=0)/ (tp.sum(axis=0)+fn.sum(axis=0))
f1=2*tp.sum(axis=0)/ (2*tp.sum(axis=0)+fp.sum(axis=0)+fn.sum(axis=0))

In [ ]:
print(precision)
print(recall)
print(f1)

**计算macro、micro形式下的三种值**

In [ ]:
macro_precision=precision.mean()
macro_recall=recall.mean()
macro_f1=f1.mean()
print(macro_precision)
print(macro_recall)
print(macro_f1)

In [ ]:
micro_precision=tp.sum()/ (tp.sum()+fp.sum())
micro_recall=tp.sum()/ (tp.sum()+fn.sum())
micro_f1=2*tp.sum()/ (2*tp.sum()+fp.sum()+fn.sum())
print(micro_precision)
print(micro_recall)
print(micro_f1)

**验证上述micro、micro形式下的三种值是否算对**

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
print(precision_score(y_true, y_pred, average='macro'))
print(recall_score(y_true, y_pred, average='macro'))
print(f1_score(y_true, y_pred, average='macro'))

In [ ]:
print(precision_score(y_true, y_pred, average='micro'))
print(recall_score(y_true, y_pred, average='micro'))
print(f1_score(y_true, y_pred, average='micro'))